## MILP tutorial 2 : make a milp model

This tutorial requires a full understanding of the previous tutorials of the serie

In this tutorial, we extend the previous problem to integer variables. 

####   Create a MILP model

As in tutorial 1, we have a load to supply using two production means. The first, can only supply a package of energy (car battery replacement for example), while the second can supply any amount of energy at a greater price. The problem properties are the following:

-   Load : 20 MW
-   Prod1 :
    - package energy : 7MWh
    - price : 1€ / package
-   Prod2 :
    - maximum power : 15MW
    - price : 5€ / MW

For energy/power consistency, we will consider that out time step is 1h.

Intuitively, the Prod1 mean is expected to provide 2 packages, and the Prod2 is expected to be used to supply the remaining load.

In [1]:
# Import the Docplex Model object that manages the problem and its resolution
from docplex.mp.model import Model
model = Model()

In that case, we do not need to set a maximum to the production_1 variable as the count of packages bought will naturally be limited by the resolution. The production_1 variable can here take any integer value above 0.

In [2]:
production_1 = model.integer_var(name="production_1", lb=0, ub=None)
production_2 = model.continuous_var(name="production_2", lb=0, ub=15)

To make a 7 MWh package from an integer, we only need to multiply it by 7

In [3]:
model += production_1 * 7 + production_2 == 20

We can now provide the objective of the model that we are willing to minimize.

In [4]:
model.set_objective("min", 1 * production_1 + 5 * production_2)

For education purposes, it can be very informative to write down the model that the solver will read, it can be done using the command:

In [5]:
model.export_as_lp("my_milp_problem.lp")

'my_milp_problem.lp'

The problem can now be solved and the results imported from the solver. We find the couple (2*7, 6) as we expected:

- We want as many batteries as possible as they are cheaper;
- We cannot provide the remaining 6 MWh with a battery as there would be an excess of 1MWh.

In [6]:
model.solve(log_output = True)

print(f"\nResults: {production_1.solution_value} * 7, {production_2.solution_value}")

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)

Results: 2.0 * 7, 6.0
